# Model Comparison Results

This notebook analyzes and visualizes the results of our software defect prediction models (SVM-RBF, Balanced Random Forest, and XGBoost).

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualization
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

## 1. Load Model Comparison Results
Load the results from our model comparison experiments.

In [ ]:
# Load results from JSON file
results_path = os.path.join('..', 'reports', 'model_comparison', 'model_comparison_results.json')

with open(results_path, 'r') as f:
    results = json.load(f)

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df.set_index('model', inplace=True)

# Display results
print("Model Comparison Results:")
display(results_df.round(3))

## 2. Visualize Model Performance
Create bar plots to compare model performance across different metrics.

In [ ]:
def plot_model_comparison(results_df):
    metrics = ["Precision", "Recall", "F-beta Score", "PR-AUC"]
    model_names = results_df.index
    n_models = len(model_names)
    n_metrics = len(metrics)
    
    # Create subplots
    fig, axes = plt.subplots(1, n_metrics, figsize=(20, 6), sharey=True)
    fig.suptitle("Model Comparison Across Metrics", fontsize=16, y=1.05)
    
    # Color palette
    colors = plt.cm.viridis(np.linspace(0, 1, n_models))
    
    # Plot each metric
    for i, metric in enumerate(metrics):
        values = results_df[metric]
        bars = axes[i].bar(range(n_models), values, color=colors)
        
        # Customize subplot
        axes[i].set_title(metric)
        axes[i].set_xticks(range(n_models))
        axes[i].set_xticklabels(model_names, rotation=45, ha='right')
        if i == 0:
            axes[i].set_ylabel('Score')
        
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            axes[i].text(bar.get_x() + bar.get_width()/2., height,
                        f'{height:.3f}',
                        ha='center', va='bottom')
    
    plt.tight_layout()
    return fig

# Create visualization
fig = plot_model_comparison(results_df)
plt.show()

## 3. Model Ranking Analysis
Rank models based on their performance across all metrics.

In [ ]:
# Calculate overall ranking based on mean performance across metrics
metrics = ["Precision", "Recall", "F-beta Score", "PR-AUC"]
rankings = results_df[metrics].rank(ascending=False)
mean_ranks = rankings.mean(axis=1).sort_values()

print("Model Rankings (lower is better):")
display(mean_ranks)

# Best performing model
best_model = mean_ranks.index[0]
print(f"\nBest Overall Model: {best_model}")
print("\nPerformance Metrics:")
display(results_df.loc[best_model].round(3))

## 4. Performance Trade-offs
Analyze the trade-offs between precision and recall for each model.

In [ ]:
# Create scatter plot of precision vs recall
plt.figure(figsize=(10, 8))
scatter = plt.scatter(results_df['Recall'], results_df['Precision'], 
                     c=results_df['F-beta Score'], cmap='viridis', 
                     s=200)

# Add labels for each point
for idx in results_df.index:
    plt.annotate(idx, (results_df.loc[idx, 'Recall'], results_df.loc[idx, 'Precision']),
                xytext=(5, 5), textcoords='offset points')

plt.colorbar(scatter, label='F-beta Score')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Trade-off by Model')
plt.grid(True)
plt.tight_layout()
plt.show()